## Notebook Template to Quickly Test Things Out

In [2]:
# General imports
import torch
import numpy as np
import os, sys
import json
from tqdm import tqdm

In [53]:
# Local imports
sys.path.insert(0, 'src')
from utils import read_json, read_lists, write_lists, prepare_device
from parse_config import ConfigParser
from data_loader import data_loaders
import model.model as module_arch

In [46]:
# Define constants, paths
config_path = 'configs/cinic10_imagenet_misc.json'
class_list_path = 'metadata/cinic-10/class_names.txt'
target_class =  5  # worst accuracy (40.914%)

In [22]:
# Load config file, models
config_json = read_json(config_path)
config = ConfigParser(config_json)

layernum = config.config['layernum']
device, device_ids = prepare_device(config['n_gpu'])
print("Read in config file from {}".format(config_path))
      
model = config.init_obj('arch', module_arch, layernum=layernum)
model.eval()
print("Initialized model from {}".format(config.config['arch']['args']['checkpoint_path']))

class_list = read_lists(class_list_path)

Read in config file from configs/cinic10_imagenet_misc.json
Initialized model from external_code/PyTorch_CIFAR10/cifar10_models/state_dicts/vgg16_bn.pt


In [17]:
train_data_loader = data_loader_args = dict(config_json["data_loader"]["args"])
train_data_loader = data_loaders.CINIC10DataLoader(
    **data_loader_args,
    return_paths=True,
    split='train')

print("Initialized train data loader")

Initialized train data loader


In [ ]:
# Function definitions

In [47]:
# Pedal to the metal!
correct_image_paths = []
correct_images = []
incorrect_image_paths = []
incorrect_images = []
incorrect_predictions = []
with torch.no_grad():
    for idx, item in enumerate(tqdm(train_data_loader)):
        image, target, path = item
        
        # Skip any batches with no examples from target class
        if (target != target_class).all():
            continue

        # Find indices where target = target class
        target_idxs = (target == target_class).nonzero()
        target_idxs = torch.squeeze(target_idxs)

        image = image[target_idxs]
        target = target[target_idxs]
        path = [path[idx] for idx in target_idxs]  # path[target_idxs]
        
        # Move data and label to GPU
        image, target = image.to(device), target.to(device)
        
        print("image shape {}".format(image.shape))
        output = model(image)
        prediction = torch.argmax(output, dim=1)
        print("output shape {}".format(output.shape))
        # Obtain indices of where model predicted correctly and incorrectly
        correct_idxs = torch.squeeze((prediction == target_class).nonzero())
        incorrect_idxs = torch.squeeze((prediction != target_class).nonzero())
        
        print(correct_idxs.shape)
        print(incorrect_idxs.shape)
        correct_image_paths += [path[idx] for idx in correct_idxs] 
        correct_images.append(image[correct_idxs])
        
        incorrect_image_paths += [path[idx] for idx in incorrect_idxs]
        incorrect_images.append(image[incorrect_idxs])
        incorrect_predictions.append(prediction[incorrect_idxs])
    
        
    
    

 50%|████████████████████▌                    | 137/274 [00:32<00:10, 13.03it/s]

image shape torch.Size([72, 3, 32, 32])
output shape torch.Size([72, 10])
torch.Size([19])
torch.Size([53])


 51%|█████████████████████                    | 141/274 [00:33<00:11, 12.06it/s]

image shape torch.Size([256, 3, 32, 32])
output shape torch.Size([256, 10])
torch.Size([128])
torch.Size([128])
image shape torch.Size([256, 3, 32, 32])
output shape torch.Size([256, 10])
torch.Size([107])
torch.Size([149])
image shape torch.Size([256, 3, 32, 32])
output shape torch.Size([256, 10])
torch.Size([145])
torch.Size([111])
image shape torch.Size([256, 3, 32, 32])
output shape torch.Size([256, 10])
torch.Size([177])
torch.Size([79])
image shape torch.Size([256, 3, 32, 32])
output shape torch.Size([256, 10])


 53%|█████████████████████▌                   | 144/274 [00:33<00:08, 14.63it/s]

...

 59%|████████████████████████▍                | 163/274 [00:34<00:07, 14.14it/s]

image shape torch.Size([256, 3, 32, 32])
output shape torch.Size([256, 10])
torch.Size([27])
torch.Size([229])
image shape torch.Size([256, 3, 32, 32])
output shape torch.Size([256, 10])
torch.Size([27])
torch.Size([229])
image shape torch.Size([256, 3, 32, 32])
output shape torch.Size([256, 10])
torch.Size([23])
torch.Size([233])
image shape torch.Size([256, 3, 32, 32])
output shape torch.Size([256, 10])


 60%|████████████████████████▋                | 165/274 [00:34<00:08, 13.31it/s]

torch.Size([35])
torch.Size([221])
image shape torch.Size([16, 3, 32, 32])
output shape torch.Size([16, 10])
torch.Size([0])
torch.Size([16])


100%|█████████████████████████████████████████| 274/274 [00:43<00:00,  6.33it/s]


In [55]:
# print(len(incorrect_image_paths))
# print(len(correct_image_paths))
correct_images = torch.cat(correct_images, dim=0)
correct_images = correct_images.cpu()
# print(correct_images.shape)
incorrect_images = torch.cat(incorrect_images, dim=0)
incorrect_images = incorrect_images.cpu()

4025
2975
torch.Size([2975, 3, 32, 32])


### Save images as checkpoints and paths as txt files

In [57]:
# Save paths to correct and incorrect images
save_dir = os.path.join(
    'metadata', 
    'CINIC10-ImageNet', 
    class_list[target_class],
    config.config['arch']['args']['type'])

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

# Save list of correct image paths and the images
correct_image_paths_filepath = os.path.join(save_dir, 'correct_image_paths.txt')
correct_images_save_path = os.path.join(save_dir, 'correct_images.pth')
write_lists(correct_image_paths_filepath, correct_image_paths)
torch.save(correct_images, correct_images_save_path)

incorrect_image_paths_filepath = os.path.join(save_dir, 'incorrect_image_paths.txt')
incorrect_images_save_path = os.path.join(save_dir, 'incorrect_images.pth')
write_lists(incorrect_image_paths_filepath, incorrect_image_paths)
torch.save(incorrect_images, incorrect_images_save_path)

